In [3]:
import os
import flopy
import pyemu
import pandas as pd
import numpy as np
from pyemu.pst.pst_utils import SFMT

flopy is installed in C:\Users\seonggyu.park\Miniconda3\envs\sm_pest\lib\site-packages\flopy


In [4]:
wd = "E:\okvg_pp\okvg_091120_pest"
os.chdir(wd)
mname = "modflow.mfn"


In [1]:
# %matplotlib inline
# import os, shutil
# import sys
# sys.path.append("..")
# import numpy as np
# from IPython.display import Image
# import pandas as pd
# import matplotlib.pyplot as plt

# import flopy as flopy
# import pyemu
# import shapefile #the pyshp module
# from pyemu.pst.pst_utils import SFMT,IFMT,FFMT,pst_config

In [5]:
m = flopy.modflow.Modflow.load(mname,model_ws=wd)
m.check()

Errors and/or Warnings encountered.
Errors and/or Warnings encountered.

modflow MODEL DATA VALIDATION SUMMARY:
  2406 Errors:
    RIV package: RIV stage below rbots
  3 Warnings:
    OC package: action(s) defined in OC stress_period_data ignored as they are not part the stress periods defined by DIS
    RCH package: Mean R/T ratio < checker warning threshold of 2e-08 for 1 stress periods
    RIV package: BC in inactive cell

  Checks that passed:
    Unit number conflicts
    Compatible solver package
    DIS package: zero or negative thickness
    DIS package: thin cells (less than checker threshold of 1.0)
    DIS package: nan values in top array
    DIS package: nan values in bottom array
    BAS6 package: isolated cells in ibound array
    BAS6 package: Not a number
    UPW package: zero or negative horizontal hydraulic conductivity values
    UPW package: zero or negative vertical hydraulic conductivity values
    UPW package: negative horizontal anisotropy values
    UPW package

### Model Setup

### - Precalibration results

<img src="figs\okvg_swatmf_ipsl_wb.png" style="float: left; width: 70%; margin-right: 1%; margin-bottom: 0em; background-color : white">
<img src="figs\okvg_swatmf_ipsl.png" style="float: left; width: 70%; margin-right: 1%; margin-bottom: 0em; background-color : white">


- Simulation Period 
    * Jan. 01, 2000 ~ Dec. 31, 2010 with 3 years warm-up period (results start at 2002) 11 years
    <br/><br/>
- Measurement Duration
    * Varies
    * Streamflow - Jul. 22, 2002 ~ Dec. 31, 2010
    * Watertable - Searching ...
    <br/><br/>
- Calibration / Validation
    * 2003 1/1 - 2007 for Calibration
    * 2008 - 2010 for validation (???)
    <br/><br/>
- Streamflow: Little baseflow, High peak, Peak early -> , peak low, shift to little right
    - SWAT parameters:
        * Little Baseflow & High Peak
            - CN2 - Decrease
            - ESCO - Increase
            - SOL_AWC - Increase
        * Peak early
            * HRU_SLP - Decrease
            * OV_N - Increase
            * SLSUBBSN - Increase (The value of overland flow length)
    - MODFLOW parameters:
        * Little Baseflow & High Peak
            - Riverbed cond - Decrease
            - Riverbed bottom elevation - Decrease
        * K - ? 
        * Sy - ?
        
- Watertable -> high watertable, slow recession, 
    - SWAT parameters:
        * RCHRG_DP - (turned off) decrease
        * CN2 - ?
        * ESCO - ? (increase)
        * SOL_AWC - decrease
    - MODFLOW parameters:
        * K - increase 
        * Sy - ?
        * EVT depth - increase
        * River Bottom - ?
        * River conductance - decrease

In [6]:
from sm_pst_pkgs import sm_pst_par, sm_pst_utils, sm_pst_stats

### 1. Create PEST input files (template / instruction)

#### 1.1. Create template files
We are going to use the *.pval and mf_river.par files for MODFLOW parameters and model.in file for SWAT parameters.

In [7]:
# pval file
pval_file = 'okvg_3000.pval'
# model.in file
model_in = 'model.in'

In [8]:
gw_par = pyemu.utils.gw_utils.modflow_pval_to_template_file(pval_file, tpl_file=None)
print(gw_par)

parnme      parval1                       tpl
parnme                                              
hk01     hk01    17.122810   ~   hk01              ~
hk02     hk02    15.732918   ~   hk02              ~
hk03     hk03  1000.000000   ~   hk03              ~
hk04     hk04  1000.000000   ~   hk04              ~
hk05     hk05   403.224070   ~   hk05              ~
hk06     hk06  1000.000000   ~   hk06              ~
sy01     sy01     0.012378   ~   sy01              ~
sy02     sy02     0.270421   ~   sy02              ~
sy03     sy03     0.030000   ~   sy03              ~
sy04     sy04     0.030000   ~   sy04              ~
sy05     sy05     0.042186   ~   sy05              ~
sy06     sy06     0.010000   ~   sy06              ~
sy07     sy07     0.247212   ~   sy07              ~
sy08     sy08     0.235688   ~   sy08              ~
hk7       hk7   611.179200   ~   hk7               ~
hk8       hk8    59.847575   ~   hk8               ~


In [9]:
# Create pval and model template files
sw_par = sm_pst_utils.model_in_to_template_file(model_in_file=model_in, tpl_file=None)

In [10]:
sw_par

,parnme,parval1,tpl
parnme,,,
"r__CH_D.rte________66,105","r__CH_D.rte________66,105",-0.336521,~ CH_D.rte________66 ~
"r__CH_D.rte________68,83","r__CH_D.rte________68,83",-0.336521,~ CH_D.rte________6 ~
"r__CH_D.rte________6,8-13,16-17,19-21,24-25,31-32,34-35,47-50,61-62,69,77,81,84,87,88,90-92,97-98,101-102,106,108-109,118,120-123,131-136,141-142,145-147","r__CH_D.rte________6,8-13,16-17,19-21,24-25,31-32,34-35,47-50,61-62,69,77,81,84,87,88,90-92,97-9...",-0.336521,"~ CH_D.rte________6,8-13,16-17,19-21,24-25,31-32,34-35,47-50,61-62,69,77,81,84,87,88,90-92,97..."
"r__CH_D.rte________1-5,7,14-15,18,22-23,26-30,33,36-46,51-60,63-65,67,70-73,75-76,78-80,82,85-86,93-96,99-100,103-104,107,110-117,119,124-130,137-140,143-144,148-152,155-156,163-164,168-170,172-181,183-194,199-201,203-208,210-217,219-222,225-227,229,232-233,235-239,241-242","r__CH_D.rte________1-5,7,14-15,18,22-23,26-30,33,36-46,51-60,63-65,67,70-73,75-76,78-80,82,85-86...",-0.526964,"~ CH_D.rte________1-5,7,14-15,18,22-23,26-30,33,36-46,51-60,63-65,67,70-73,75-76,78-80,82,85-..."
"r__CH_D.rte________153-154,157,159-162,165-167,171,182,195-198,202,209,218,223-224,228,230-231,234,240","r__CH_D.rte________153-154,157,159-162,165-167,171,182,195-198,202,209,218,223-224,228,230-231,2...",1.799610,"~ CH_D.rte________153-154,157,159-162,165-167,171,182,195-198,202,209,218,223-224,228,230-231..."
r__CH_D.rte________243-257,r__CH_D.rte________243-257,0.028900,~ CH_D.rte________243 ~
"v__CH_K2.rte________6,8-13,16-17,19-21,24-25,31-32,34-35,47-50,61-62,69,77,81,84,87,88,90-92,97-98,101-102,106,108-109,118,120-123,131-136,141-142,145-147","v__CH_K2.rte________6,8-13,16-17,19-21,24-25,31-32,34-35,47-50,61-62,69,77,81,84,87,88,90-92,97-...",5.000000,"~ CH_K2.rte________6,8-13,16-17,19-21,24-25,31-32,34-35,47-50,61-62,69,77,81,84,87,88,90-92,9..."
"v__CH_K2.rte________66,105","v__CH_K2.rte________66,105",5.000000,~ CH_K2.rte________66 ~
"v__CH_K2.rte________68,83","v__CH_K2.rte________68,83",5.000000,~ CH_K2.rte________6 ~


### Name parameters

In [11]:

subnams = ['66', '68', '147', '227', '240', '243']
parnams = ['chd', 'chk', 'cn', 'es', 'awc', 'ovn', 'slp', 'chw', 'chs', 'chn']

count = 0
for i in parnams:
    for j in subnams:
        nam = ('{}_{}'.format(i, j))
        sw_par.iloc[count, 2] = " ~   {0:15s}   ~".format(nam)
        count += 1



In [12]:
with open('model.in.tpl', 'w') as f:
    f.write("ptf ~\n")
    # f.write("{0:10d} #NP\n".format(mod_df.shape[0]))
    SFMT_LONG = lambda x: "{0:<280s} ".format(str(x))
    f.write(sw_par.loc[:, ["parnme", "tpl"]].to_string(
                                                    col_space=0,
                                                    formatters=[SFMT_LONG, SFMT],
                                                    index=False,
                                                    header=False,
                                                    justify="left"))

In [22]:
print(sw_par)

4,157,159-162,165-167,171,182,195-198,202,209,218,223-224,228,230-231,...   
v__CH_N2.rte________243-257                                                                                                                                                   v__CH_N2.rte________243-257   

                                                                                                       parval1  \
parnme                                                                                                           
r__CH_D.rte________66,105                                                                            -0.336521   
r__CH_D.rte________68,83                                                                             -0.336521   
r__CH_D.rte________6,8-13,16-17,19-21,24-25,31-32,34-35,47-50,61-62,69,77,81,84,87,88,90-92,97-98... -0.336521   
r__CH_D.rte________1-5,7,14-15,18,22-23,26-30,33,36-46,51-60,63-65,67,70-73,75-76,78-80,82,85-86,... -0.526964   
r__CH_D.rte________153-154,157,15

### 1.1.1 Create river parameters

In [13]:
# provide channel ids that will be used for calibration
subs = ['g147', 'g225', 'g240', 'g244']
sm_pst_par.create_riv_par(wd, subs)

'mf_riv.par' file has been exported to the SWAT-MODFLOW working directory!


,parnme,chg_type,val
parnme,,,
rivcd_g147,rivcd_g147,unfchg,0.001
rivcd_g225,rivcd_g225,unfchg,0.001
rivcd_g240,rivcd_g240,unfchg,0.001
rivcd_g244,rivcd_g244,unfchg,0.001
rivbot_g147,rivbot_g147,unfchg,0.001
rivbot_g225,rivbot_g225,unfchg,0.001
rivbot_g240,rivbot_g240,unfchg,0.001
rivbot_g244,rivbot_g244,unfchg,0.001


In [43]:
# create a template file for mf_riv.par file
sm_pst_utils.riv_par_to_template_file('mf_riv.par')

,parnme,chg_type,parval1,tpl
parnme,,,,
rivcd_g147,rivcd_g147,pctchg,0.001,~ rivcd_g147 ~
rivcd_g225,rivcd_g225,pctchg,0.001,~ rivcd_g225 ~
rivcd_g240,rivcd_g240,pctchg,0.001,~ rivcd_g240 ~
rivcd_g244,rivcd_g244,pctchg,0.001,~ rivcd_g244 ~
rivbot_g147,rivbot_g147,unfchg,0.001,~ rivbot_g147 ~
rivbot_g225,rivbot_g225,unfchg,0.001,~ rivbot_g225 ~
rivbot_g240,rivbot_g240,unfchg,0.001,~ rivbot_g240 ~
rivbot_g244,rivbot_g244,unfchg,0.001,~ rivbot_g244 ~


In [17]:
# overwrite the river package file
sm_pst_par.riv_par(wd)

The "riv_package.org" file already exists...
okvg_3000.riv file is overwritten successfully!


## 1.2. Build instruction files (streamflow / watertable / baseflow)
### 1.2.1. Streamflow (output.rch)

In [15]:
# file path
rch_file = 'output.rch'
# reach numbers that are used for calibration
subs = [225, 240]

In [16]:
# extract month_streamflow
sm_pst_utils.extract_month_str(rch_file, subs, '1/1/2003', '1/1/2003', '12/31/2007')

cha_225.txt file has been created...
cha_240.txt file has been created...
Finished ...


In [17]:
sm_pst_utils.extract_month_str?

Signature:
sm_pst_utils.extract_month_str(
    rch_file,
    channels,
    start_day,
    cali_start_day,
    cali_end_day,
)
Docstring:
extract a simulated streamflow from the output.rch file,
   store it in each channel file.

Args:
    - rch_file (`str`): the path and name of the existing output file
    - channels (`list`): channel number in a list, e.g. [9, 60]
    - start_day ('str'): simulation start day after warm period, e.g. '1/1/1985'
    - end_day ('str'): simulation end day e.g. '12/31/2005'

Example:
    sm_pst_utils.extract_month_str('path', [9, 60], '1/1/1993', '1/1/1993', '12/31/2000')
File:      e:\okvg_pp\sm_pst_pkgs\sm_pst_utils.py
Type:      function


### 1.2.3. Create instruction files for each str_sim file using the 'streamflow.obd' file

In [18]:
# because we have 3 streamgages let's loop for them
# read streamobd and get column names
stf_obd = pd.read_csv(
                    'streamflow.obd',
                    sep='\t',
                    index_col=0,
                    parse_dates=True,
                    na_values=[-999, '']
                    )
# stf_obd_c = stf_obd.resample('M').mean()
# stf_obd_c.to_csv('streamflow_m.obd', sep='\t', na_rep=-999, float_format='%.2f')
print(stf_obd.columns)
sim_files = ['cha_{:03d}.txt'.format(x) for x in subs]
print(sim_files)

Index(['sub_225', 'sub_240'], dtype='object')
['cha_225.txt', 'cha_240.txt']


In [19]:
# create instruction files for each sim file
for i in range(len(sim_files)):
    sm_pst_utils.str_obd_to_ins(sim_files[i], stf_obd.columns[i], '1/1/2003', '12/31/2007')

cha_225.txt.ins file has been created...
cha_240.txt.ins file has been created...


In [20]:
sm_pst_utils.str_obd_to_ins?

Signature: sm_pst_utils.str_obd_to_ins(srch_file, col_name, start_day, end_day)
Docstring:
extract a simulated streamflow from the output.rch file,
   store it in each channel file.

Args:
    - rch_file (`str`): the path and name of the existing output file
    - channels (`list`): channel number in a list, e.g. [9, 60]
    - start_day ('str'): simulation start day after warm period, e.g. '1/1/1993'
    - end_day ('str'): simulation end day e.g. '12/31/2000'

Example:
    pest_utils.extract_month_str('path', [9, 60], '1/1/1993', '12/31/2000')
File:      d:\projects\watersheds\okavango\analysis\swat-modflows\sm_pst_pkgs\sm_pst_utils.py
Type:      function


In [17]:
'''
# We don't have watertable data
sm_pst_utils.extract_watertable_sim([5699, 5832], '1/1/1980', '12/31/2005')
sm_pst_utils.mf_obd_to_ins('wt_5832.txt', 'g_5832', '1/1/1980', '12/31/2005')
'''

"\n# We don't have watertable data\nsm_pst_utils.extract_watertable_sim([5699, 5832], '1/1/1980', '12/31/2005')\nsm_pst_utils.mf_obd_to_ins('wt_5832.txt', 'g_5832', '1/1/1980', '12/31/2005')\n"

# Create baseflow rate instruction file

### Extract avarage basflow ratio from monthly simulated stream dicharges

In [20]:
sm_pst_utils.extract_month_baseflow('output.sub', [66, 68, 147], '1/1/2003', '1/1/2003', '12/31/2007')

Average baseflow rate for 066 has been calculated ...
Average baseflow rate for 068 has been calculated ...
Average baseflow rate for 147 has been calculated ...
Finished ...



## Create a dummy pst file 

In [21]:
io_files = pyemu.helpers.parse_dir_for_io_files('.')
pst = pyemu.Pst.from_io_files(*io_files)
pyemu.helpers.pst_from_io_files(io_files[0], io_files[1], io_files[2], io_files[3], 'okvg_dummy.pst')

# print(os.chdir(".."))
io_files

error using inschek for instruction file baseflow_ratio.out.ins:run() returned non-zero: 1
observations in this instruction file will havegeneric values.
error using inschek for instruction file cha_225.txt.ins:run() returned non-zero: 1
observations in this instruction file will havegeneric values.
error using inschek for instruction file cha_240.txt.ins:run() returned non-zero: 1
observations in this instruction file will havegeneric values.
error using inschek for instruction file baseflow_ratio.out.ins:run() returned non-zero: 1
observations in this instruction file will havegeneric values.
error using inschek for instruction file cha_225.txt.ins:run() returned non-zero: 1
observations in this instruction file will havegeneric values.
error using inschek for instruction file cha_240.txt.ins:run() returned non-zero: 1
observations in this instruction file will havegeneric values.
noptmax:30, npar_adj:84, nnz_obs:123


(['mf_riv.par.tpl', 'model.in.tpl', 'okvg_3000.pval.tpl'],
 ['mf_riv.par', 'model.in', 'okvg_3000.pval'],
 ['baseflow_ratio.out.ins', 'cha_225.txt.ins', 'cha_240.txt.ins'],
 ['baseflow_ratio.out', 'cha_225.txt', 'cha_240.txt'])

The ``parse_dir_for_io_files()`` helper is looking for files with the ".tpl" and ".ins" extension.  This assumes that the corresponding model input and model output files are the same name, minus the ".tpl" and ".ins" extension, respectively.  These file lists are then passed to another helper, which builds a basic control file for you (``Pst.from_io_files()``).  Let's look at this generic ``Pst`` instance:

In [22]:
par = pst.parameter_data
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
awc_147,awc_147,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
awc_227,awc_227,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
awc_240,awc_240,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
awc_243,awc_243,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
awc_66,awc_66,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...
sy04,sy04,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
sy05,sy05,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
sy06,sy06,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
sy07,sy07,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1


#### 2.1. Change parameter group name

In [23]:
for i in range(len(par)):
    if (par.iloc[i, 0][:2]) == 'sy':
        par.iloc[i, 6] = 'sy'
    elif par.iloc[i, 0][:7] == 'rivbot_':
        par.iloc[i, 6] = 'rivbot'
    elif par.iloc[i, 0][:6] == 'rivcd_':
        par.iloc[i, 6] = 'rivcd'
    elif par.iloc[i, 0][:2] == 'hk':
        par.iloc[i, 6] = 'hk'
    elif par.iloc[i, 0][:3] == 'bfr':
        par.iloc[i, 6] = 'bfr'
    else:
        par.iloc[i, 6] = 'str'
print(par)

parnme partrans parchglim  parval1       parlbnd       parubnd  \
awc_147  awc_147      log    factor      1.0  1.100000e-10  1.100000e+10   
awc_227  awc_227      log    factor      1.0  1.100000e-10  1.100000e+10   
awc_240  awc_240      log    factor      1.0  1.100000e-10  1.100000e+10   
awc_243  awc_243      log    factor      1.0  1.100000e-10  1.100000e+10   
awc_66    awc_66      log    factor      1.0  1.100000e-10  1.100000e+10   
...          ...      ...       ...      ...           ...           ...   
sy04        sy04      log    factor      1.0  1.100000e-10  1.100000e+10   
sy05        sy05      log    factor      1.0  1.100000e-10  1.100000e+10   
sy06        sy06      log    factor      1.0  1.100000e-10  1.100000e+10   
sy07        sy07      log    factor      1.0  1.100000e-10  1.100000e+10   
sy08        sy08      log    factor      1.0  1.100000e-10  1.100000e+10   

        pargp  scale  offset  dercom  
awc_147   str    1.0     0.0       1  
awc_227   str    1.

#### 2.2. Set par ranges and initial values for parameters

### 2.2.3. MODFLOW

### Let's start values from 1st round of calibration result

In [24]:
# for MODFLOW parameters
count = 0
for i in range(len(par)):
    if (par.iloc[i, 6] == 'hk') and (par.iloc[i, 0] == 'hk01'):  # hk_sed
        par.iloc[i, 3] = 5.2502812E+01      
        par.iloc[i, 4] = 1.000000e+00
        par.iloc[i, 5] = 1.000000e+02
    elif (par.iloc[i, 6] == 'hk') and (par.iloc[i, 0] == 'hk02'):  # hk_as01
        par.iloc[i, 3] = 1.500000e+01       
        par.iloc[i, 4] = 1.500000e+00
        par.iloc[i, 5] = 1.500000e+02  
    elif (par.iloc[i, 6] == 'hk') and (par.iloc[i, 0] == 'hk03'):  # hk_as02
        par.iloc[i, 3] = 2.0219147E+00      
        par.iloc[i, 4] = 1.000000e+00
        par.iloc[i, 5] = 1.000000e+02    
    elif (par.iloc[i, 6] == 'hk') and (par.iloc[i, 0] == 'hk04'):  # hk_as03
        par.iloc[i, 3] = 1.7352222E+02     
        par.iloc[i, 4] = 5.000000e+00
        par.iloc[i, 5] = 5.000000e+02    
    elif (par.iloc[i, 6] == 'hk') and (par.iloc[i, 0] == 'hk05'):  # hk_org
        par.iloc[i, 3] = 500       
        par.iloc[i, 4] = 1.000000e+01
        par.iloc[i, 5] = 1.000000e+03    
    elif (par.iloc[i, 6] == 'hk') and (par.iloc[i, 0] == 'hk06'):  # hk_af01
        par.iloc[i, 3] = 800      
        par.iloc[i, 4] = 1.500000e+01
        par.iloc[i, 5] = 1.500000e+03    
    elif (par.iloc[i, 6] == 'hk') and (par.iloc[i, 0] == 'hk7'):  # hk_af02
        par.iloc[i, 3] = 1000       
        par.iloc[i, 4] = 5.000000e+01
        par.iloc[i, 5] = 5.000000e+03
    elif (par.iloc[i, 6] == 'hk') and (par.iloc[i, 0] == 'hk8'):  # hk_af02
        par.iloc[i, 3] = 7.9413064E+01       
        par.iloc[i, 4] = 1.000000e+00
        par.iloc[i, 5] = 1.000000e+03
    elif (par.iloc[i, 6] == 'sy') and (par.iloc[i, 0] == 'sy01'):  # sy_sed
        par.iloc[i, 3] = 2.8550744E-02      
        par.iloc[i, 4] = 1.000000e-03
        par.iloc[i, 5] = 5.000000e-01
    elif (par.iloc[i, 6] == 'sy') and (par.iloc[i, 0] == 'sy02'):  # sy_as01
        par.iloc[i, 3] = 1.5129263E-01       
        par.iloc[i, 4] = 1.000000e-03
        par.iloc[i, 5] = 5.000000e-01
    elif (par.iloc[i, 6] == 'sy') and (par.iloc[i, 0] == 'sy03'):  # sy_as02
        par.iloc[i, 3] = 2.4738231E-01      
        par.iloc[i, 4] = 1.000000e-03
        par.iloc[i, 5] = 5.000000e-01   
    elif (par.iloc[i, 6] == 'sy') and (par.iloc[i, 0] == 'sy04'):  # sy_as03
        par.iloc[i, 3] = 4.0476304E-01     
        par.iloc[i, 4] = 1.000000e-03
        par.iloc[i, 5] = 5.000000e-01   
    elif (par.iloc[i, 6] == 'sy') and (par.iloc[i, 0] == 'sy05'):  # sy_org
        par.iloc[i, 3] = 2.8398034E-02      
        par.iloc[i, 4] = 1.000000e-03
        par.iloc[i, 5] = 5.000000e-01  
    elif (par.iloc[i, 6] == 'sy') and (par.iloc[i, 0] == 'sy06'):  # sy_af01
        par.iloc[i, 3] = 1.9769060E-01      
        par.iloc[i, 4] = 1.000000e-03
        par.iloc[i, 5] = 5.000000e-01  
    elif (par.iloc[i, 6] == 'sy') and (par.iloc[i, 0] == 'sy07'):  # sy_af02
        par.iloc[i, 3] = 4.4609847E-01       
        par.iloc[i, 4] = 1.000000e-03
        par.iloc[i, 5] = 5.000000e-01
    elif (par.iloc[i, 6] == 'sy') and (par.iloc[i, 0] == 'sy08'):  # sy_af02
        par.iloc[i, 3] = 1.9699275E-01       
        par.iloc[i, 4] = 1.000000e-03
        par.iloc[i, 5] = 5.000000e-01
    else:
        count += 1
print(count)

68


In [30]:
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
awc_147,awc_147,log,factor,1.000000,1.100000e-10,1.100000e+10,str,1.0,0.0,1
awc_227,awc_227,log,factor,1.000000,1.100000e-10,1.100000e+10,str,1.0,0.0,1
awc_240,awc_240,log,factor,1.000000,1.100000e-10,1.100000e+10,str,1.0,0.0,1
awc_243,awc_243,log,factor,1.000000,1.100000e-10,1.100000e+10,str,1.0,0.0,1
awc_66,awc_66,log,factor,1.000000,1.100000e-10,1.100000e+10,str,1.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...
sy04,sy04,log,factor,0.404763,1.000000e-03,5.000000e-01,sy,1.0,0.0,1
sy05,sy05,log,factor,0.028398,1.000000e-03,5.000000e-01,sy,1.0,0.0,1
sy06,sy06,log,factor,0.197691,1.000000e-03,5.000000e-01,sy,1.0,0.0,1
sy07,sy07,log,factor,0.446098,1.000000e-03,5.000000e-01,sy,1.0,0.0,1


- K

In [22]:
'''
par.loc['kc', 'parval1'] = 0.2592432
par.loc['kc', 'parlbnd'] = 0.2592432e-02
par.loc['kc', 'parubnd'] = 0.2592432e+02
par.loc['kc', 'offset'] = 0

par.loc['kdfdc', 'parval1'] = 0.203472
par.loc['kdfdc', 'parlbnd'] = 0.203472e-02
par.loc['kdfdc', 'parubnd'] = 0.203472e+02
par.loc['kdfdc', 'offset'] = 0

par.loc['kked', 'parval1'] = 0.5184
par.loc['kked', 'parlbnd'] = 0.5184e-02
par.loc['kked', 'parubnd'] = 0.5184e+02
par.loc['kked', 'offset'] = 0

par.loc['kms', 'parval1'] = 0.1296
par.loc['kms', 'parlbnd'] = 0.1296e-02
par.loc['kms', 'parubnd'] = 0.1296e+02
par.loc['kms', 'offset'] = 0

par.loc['kpw', 'parval1'] = 0.2592432
par.loc['kpw', 'parlbnd'] = 0.2592432e-02
par.loc['kpw', 'parubnd'] = 0.2592432e+02
par.loc['kpw', 'offset'] = 0

par.loc['qal', 'parval1'] = 296
par.loc['qal', 'parlbnd'] = 296e-01
par.loc['qal', 'parubnd'] = 1000
par.loc['qal', 'offset'] = 0

par.loc['qt', 'parval1'] = 596
par.loc['qt', 'parlbnd'] = 596e-01
par.loc['qt', 'parubnd'] = 1500
par.loc['qt', 'offset'] = 0

par.loc['ed', 'parval1'] = 10
par.loc['ed', 'parlbnd'] = 10e-01
par.loc['ed', 'parubnd'] = 10e+01
par.loc['ed', 'offset'] = 0
'''


"\npar.loc['kc', 'parval1'] = 0.2592432\npar.loc['kc', 'parlbnd'] = 0.2592432e-02\npar.loc['kc', 'parubnd'] = 0.2592432e+02\npar.loc['kc', 'offset'] = 0\n\npar.loc['kdfdc', 'parval1'] = 0.203472\npar.loc['kdfdc', 'parlbnd'] = 0.203472e-02\npar.loc['kdfdc', 'parubnd'] = 0.203472e+02\npar.loc['kdfdc', 'offset'] = 0\n\npar.loc['kked', 'parval1'] = 0.5184\npar.loc['kked', 'parlbnd'] = 0.5184e-02\npar.loc['kked', 'parubnd'] = 0.5184e+02\npar.loc['kked', 'offset'] = 0\n\npar.loc['kms', 'parval1'] = 0.1296\npar.loc['kms', 'parlbnd'] = 0.1296e-02\npar.loc['kms', 'parubnd'] = 0.1296e+02\npar.loc['kms', 'offset'] = 0\n\npar.loc['kpw', 'parval1'] = 0.2592432\npar.loc['kpw', 'parlbnd'] = 0.2592432e-02\npar.loc['kpw', 'parubnd'] = 0.2592432e+02\npar.loc['kpw', 'offset'] = 0\n\npar.loc['qal', 'parval1'] = 296\npar.loc['qal', 'parlbnd'] = 296e-01\npar.loc['qal', 'parubnd'] = 1000\npar.loc['qal', 'offset'] = 0\n\npar.loc['qt', 'parval1'] = 596\npar.loc['qt', 'parlbnd'] = 596e-01\npar.loc['qt', 'parubn

- Sy

In [23]:
'''
par.loc['kc_sy', 'parval1'] = 0.18
par.loc['kc_sy', 'parlbnd'] = 0.18e-01
par.loc['kc_sy', 'parubnd'] = 0.3
par.loc['kc_sy', 'offset'] = 0

par.loc['kdfdc_sy', 'parval1'] = 0.05
par.loc['kdfdc_sy', 'parlbnd'] = 0.05e-01
par.loc['kdfdc_sy', 'parubnd'] = 0.15
par.loc['kdfdc_sy', 'offset'] = 0

par.loc['kked_sy', 'parval1'] = 0.12
par.loc['kked_sy', 'parlbnd'] = 0.12e-01
par.loc['kked_sy', 'parubnd'] = 0.25
par.loc['kked_sy', 'offset'] = 0

par.loc['kms_sy', 'parval1'] = 0.05
par.loc['kms_sy', 'parlbnd'] = 0.05e-01
par.loc['kms_sy', 'parubnd'] = 0.15
par.loc['kms_sy', 'offset'] = 0

par.loc['kpw_sy', 'parval1'] = 0.18
par.loc['kpw_sy', 'parlbnd'] = 0.18e-01
par.loc['kpw_sy', 'parubnd'] = 0.3
par.loc['kpw_sy', 'offset'] = 0


par.loc['qal_sy', 'parval1'] = 0.15
par.loc['qal_sy', 'parlbnd'] = 0.1
par.loc['qal_sy', 'parubnd'] = 0.5
par.loc['qal_sy', 'offset'] = 0

par.loc['qt_sy', 'parval1'] = 0.15
par.loc['qt_sy', 'parlbnd'] = 0.1
par.loc['qt_sy', 'parubnd'] = 0.5
par.loc['qt_sy', 'offset'] = 0

par.loc['ed_sy', 'parval1'] = 0.25
par.loc['ed_sy', 'parlbnd'] = 0.15
par.loc['ed_sy', 'parubnd'] = 0.5
par.loc['ed_sy', 'offset'] = 0

'''

"\npar.loc['kc_sy', 'parval1'] = 0.18\npar.loc['kc_sy', 'parlbnd'] = 0.18e-01\npar.loc['kc_sy', 'parubnd'] = 0.3\npar.loc['kc_sy', 'offset'] = 0\n\npar.loc['kdfdc_sy', 'parval1'] = 0.05\npar.loc['kdfdc_sy', 'parlbnd'] = 0.05e-01\npar.loc['kdfdc_sy', 'parubnd'] = 0.15\npar.loc['kdfdc_sy', 'offset'] = 0\n\npar.loc['kked_sy', 'parval1'] = 0.12\npar.loc['kked_sy', 'parlbnd'] = 0.12e-01\npar.loc['kked_sy', 'parubnd'] = 0.25\npar.loc['kked_sy', 'offset'] = 0\n\npar.loc['kms_sy', 'parval1'] = 0.05\npar.loc['kms_sy', 'parlbnd'] = 0.05e-01\npar.loc['kms_sy', 'parubnd'] = 0.15\npar.loc['kms_sy', 'offset'] = 0\n\npar.loc['kpw_sy', 'parval1'] = 0.18\npar.loc['kpw_sy', 'parlbnd'] = 0.18e-01\npar.loc['kpw_sy', 'parubnd'] = 0.3\npar.loc['kpw_sy', 'offset'] = 0\n\n\npar.loc['qal_sy', 'parval1'] = 0.15\npar.loc['qal_sy', 'parlbnd'] = 0.1\npar.loc['qal_sy', 'parubnd'] = 0.5\npar.loc['qal_sy', 'offset'] = 0\n\npar.loc['qt_sy', 'parval1'] = 0.15\npar.loc['qt_sy', 'parlbnd'] = 0.1\npar.loc['qt_sy', 'parubn

- SWAT

In [31]:
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
awc_147,awc_147,log,factor,1.000000,1.100000e-10,1.100000e+10,str,1.0,0.0,1
awc_227,awc_227,log,factor,1.000000,1.100000e-10,1.100000e+10,str,1.0,0.0,1
awc_240,awc_240,log,factor,1.000000,1.100000e-10,1.100000e+10,str,1.0,0.0,1
awc_243,awc_243,log,factor,1.000000,1.100000e-10,1.100000e+10,str,1.0,0.0,1
awc_66,awc_66,log,factor,1.000000,1.100000e-10,1.100000e+10,str,1.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...
sy04,sy04,log,factor,0.404763,1.000000e-03,5.000000e-01,sy,1.0,0.0,1
sy05,sy05,log,factor,0.028398,1.000000e-03,5.000000e-01,sy,1.0,0.0,1
sy06,sy06,log,factor,0.197691,1.000000e-03,5.000000e-01,sy,1.0,0.0,1
sy07,sy07,log,factor,0.446098,1.000000e-03,5.000000e-01,sy,1.0,0.0,1


- chd:
    - val: 1.001
    - lbnd: 0.001
    - ubnd: 10
    - offset: -1

In [25]:
print(par.iloc[:, 0][:3] == 'chd')

awc_147    False
awc_227    False
awc_240    False
Name: parnme, dtype: bool


In [26]:
# CHD
for i in range(len(par)):
    if (par.iloc[i, 0][:3]) == 'chd':
        par.iloc[i, 3] = 1.001
        par.iloc[i, 4] = 0.001
        par.iloc[i, 5] = 10
        par.iloc[i, 8] = -1
# CHK
for i in range(len(par)):
    if (par.iloc[i, 0][:3]) == 'chk':
        par.iloc[i, 3] = 0.001
        par.iloc[i, 4] = 0.0001
        par.iloc[i, 5] = 10
        # par.iloc[i, 8] = -1
# ESCO
for i in range(len(par)):
    if (par.iloc[i, 0][:3]) == 'es_':
        par.iloc[i, 3] = 1.001
        par.iloc[i, 4] = 0.1
        par.iloc[i, 5] = 1.9
        par.iloc[i, 8] = -1
# AWC
for i in range(len(par)):
    if (par.iloc[i, 0][:3]) == 'awc':
        par.iloc[i, 3] = 1.001
        par.iloc[i, 4] = 0.1
        par.iloc[i, 5] = 1.9
        par.iloc[i, 8] = -1
# OVN
for i in range(len(par)):
    if (par.iloc[i, 0][:3]) == 'ovn':
        par.iloc[i, 3] = 1.001
        par.iloc[i, 4] = 0.1
        par.iloc[i, 5] = 1000
        par.iloc[i, 8] = -1
# SLP
for i in range(len(par)):
    if (par.iloc[i, 0][:3]) == 'slp':
        par.iloc[i, 3] = 1.001
        par.iloc[i, 4] = 0.1
        par.iloc[i, 5] = 100
        par.iloc[i, 8] = -1
# CHW
for i in range(len(par)):
    if (par.iloc[i, 0][:3]) == 'chw':
        par.iloc[i, 3] = 1.001
        par.iloc[i, 4] = 0.1
        par.iloc[i, 5] = 1.9
        par.iloc[i, 8] = -1
# CHN
for i in range(len(par)):
    if (par.iloc[i, 0][:3]) == 'chn':
        par.iloc[i, 3] = 0.29
        par.iloc[i, 4] = 0.1
        par.iloc[i, 5] = 0.3



In [27]:
### chs
par.loc['chs_66', 'parval1'] = 0.15
par.loc['chs_66', 'parlbnd'] = 0.001
par.loc['chs_66', 'parubnd'] = 0.5
par.loc['chs_66', 'offset'] = -1

par.loc['chs_68', 'parval1'] = 0.15
par.loc['chs_68', 'parlbnd'] = 0.001
par.loc['chs_68', 'parubnd'] = 0.5
par.loc['chs_68', 'offset'] = -1

par.loc['chs_147', 'parval1'] = 0.15
par.loc['chs_147', 'parlbnd'] = 0.001
par.loc['chs_147', 'parubnd'] = 0.5
par.loc['chs_147', 'offset'] = -1

par.loc['chs_227', 'parval1'] = 0.15
par.loc['chs_227', 'parlbnd'] = 0.001
par.loc['chs_227', 'parubnd'] = 0.5
par.loc['chs_227', 'offset'] = -1

par.loc['chs_240', 'parval1'] = 0.001
par.loc['chs_240', 'parlbnd'] = 0.0001
par.loc['chs_240', 'parubnd'] = 0.1
par.loc['chs_240', 'offset'] = -1

par.loc['chs_243', 'parval1'] = 0.001
par.loc['chs_243', 'parlbnd'] = 0.0001
par.loc['chs_243', 'parubnd'] = 0.5
par.loc['chs_243', 'offset'] = -1


### CN
# cn_147

par.loc['cn_66', 'parval1'] = 1.001
par.loc['cn_66', 'parlbnd'] = 0.5
par.loc['cn_66', 'parubnd'] = 1.5
par.loc['cn_66', 'offset'] = -1

par.loc['cn_68', 'parval1'] = 1.001
par.loc['cn_68', 'parlbnd'] = 0.5
par.loc['cn_68', 'parubnd'] = 1.5
par.loc['cn_68', 'offset'] = -1

par.loc['cn_147', 'parval1'] = 1.001
par.loc['cn_147', 'parlbnd'] = 0.5
par.loc['cn_147', 'parubnd'] = 1.5
par.loc['cn_147', 'offset'] = -1

# cn_227
par.loc['cn_227', 'parval1'] = 0.9
par.loc['cn_227', 'parlbnd'] = 0.5
par.loc['cn_227', 'parubnd'] = 1.5
par.loc['cn_227', 'offset'] = -1

# cn_240
par.loc['cn_240', 'parval1'] = 1.35
par.loc['cn_240', 'parlbnd'] = 0.5
par.loc['cn_240', 'parubnd'] = 1.5
par.loc['cn_240', 'offset'] = -1

# cn_243
par.loc['cn_243', 'parval1'] = 0.9
par.loc['cn_243', 'parlbnd'] = 0.5
par.loc['cn_243', 'parubnd'] = 1.5
par.loc['cn_243', 'offset'] = -1



In [28]:
par.loc[par['pargp'] == 'rivcd']

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
rivcd_g147,rivcd_g147,log,factor,1.0,1.100000e-10,1.100000e+10,rivcd,1.0,0.0,1
rivcd_g225,rivcd_g225,log,factor,1.0,1.100000e-10,1.100000e+10,rivcd,1.0,0.0,1
rivcd_g240,rivcd_g240,log,factor,1.0,1.100000e-10,1.100000e+10,rivcd,1.0,0.0,1
rivcd_g244,rivcd_g244,log,factor,1.0,1.100000e-10,1.100000e+10,rivcd,1.0,0.0,1


# - River Parameters

In [29]:
# Let's use unfchg for riv_bot
# set +- 100 meters to default ranges and 0.001 for initials
for i in range(len(par)):
    if (par.iloc[i, 6]) == 'rivbot':  # rivbot 
        par.iloc[i, 3] = 30.1   # initial    
        par.iloc[i, 4] = 0.1   # lower
        par.iloc[i, 5] = 60   # upper
        par.iloc[i, 8] = -30   # offset

# Distiguish channels between high and low baseflow
# for low baseflow, set -500 min, 100 max, and -300 for initals
# lowbases = ['g240']
# for i in lowbases:
#     par.loc['rivbot_{}'.format(i), 'parval1'] = 600.1
#     par.loc['rivbot_{}'.format(i), 'parlbnd'] = 100
#     par.loc['rivbot_{}'.format(i), 'parubnd'] = 700
#     par.loc['rivbot_{}'.format(i), 'offset'] = -600
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
awc_147,awc_147,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
awc_227,awc_227,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
awc_240,awc_240,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
awc_243,awc_243,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
awc_66,awc_66,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
...,...,...,...,...,...,...,...,...,...,...
sy04,sy04,log,factor,0.404763,0.001,0.5,sy,1.0,0.0,1
sy05,sy05,log,factor,0.028398,0.001,0.5,sy,1.0,0.0,1
sy06,sy06,log,factor,0.197691,0.001,0.5,sy,1.0,0.0,1
sy07,sy07,log,factor,0.446098,0.001,0.5,sy,1.0,0.0,1


In [30]:
# Let's use pctchg for rivcd
# set +- 50 % to default ranges and 0.001 for initials
for i in range(len(par)):
    if (par.iloc[i, 6]) == 'rivcd':  # rivbot 
        par.iloc[i, 3] = 50.01   # initial    
        par.iloc[i, 4] = 0.1   # lower
        par.iloc[i, 5] = 100   # upper
        par.iloc[i, 8] = -50   # offset
# decrease in peak and increase in baseflow can be accomplished by decreasing river conductance
# for H peaks and L bases, set inital -20, max 10, min -50
# HpLb = [124, 92, 147, 66, 138, 228, 79]
# for i in lowbases:
#     par.loc['rivcd_{}'.format(i), 'parval1'] = 50.001
#     par.loc['rivcd_{}'.format(i), 'parlbnd'] = 0.001
#     par.loc['rivcd_{}'.format(i), 'parubnd'] = 60
#     par.loc['rivcd_{}'.format(i), 'offset'] = -50
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
awc_147,awc_147,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
awc_227,awc_227,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
awc_240,awc_240,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
awc_243,awc_243,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
awc_66,awc_66,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
...,...,...,...,...,...,...,...,...,...,...
sy04,sy04,log,factor,0.404763,0.001,0.5,sy,1.0,0.0,1
sy05,sy05,log,factor,0.028398,0.001,0.5,sy,1.0,0.0,1
sy06,sy06,log,factor,0.197691,0.001,0.5,sy,1.0,0.0,1
sy07,sy07,log,factor,0.446098,0.001,0.5,sy,1.0,0.0,1


## Observation

In [31]:
# set observation group
obd = pst.observation_data
obd

,obsnme,obsval,weight,obgnme
bfr_066,bfr_066,1.000000e+10,1.0,obgnme
bfr_068,bfr_068,1.000000e+10,1.0,obgnme
bfr_147,bfr_147,1.000000e+10,1.0,obgnme
sub_225_200301,sub_225_200301,1.000000e+10,1.0,obgnme
sub_225_200302,sub_225_200302,1.000000e+10,1.0,obgnme
...,...,...,...,...
sub_240_200708,sub_240_200708,1.000000e+10,1.0,obgnme
sub_240_200709,sub_240_200709,1.000000e+10,1.0,obgnme
sub_240_200710,sub_240_200710,1.000000e+10,1.0,obgnme
sub_240_200711,sub_240_200711,1.000000e+10,1.0,obgnme


In [32]:
# Change obd group name
for i in range(len(obd)):
    obd.iloc[i, 3] = obd.iloc[i, 0][:-7]
obd

,obsnme,obsval,weight,obgnme
bfr_066,bfr_066,1.000000e+10,1.0,bfr_
bfr_068,bfr_068,1.000000e+10,1.0,bfr_
bfr_147,bfr_147,1.000000e+10,1.0,bfr
sub_225_200301,sub_225_200301,1.000000e+10,1.0,sub_225
sub_225_200302,sub_225_200302,1.000000e+10,1.0,sub_225
...,...,...,...,...
sub_240_200708,sub_240_200708,1.000000e+10,1.0,sub_240
sub_240_200709,sub_240_200709,1.000000e+10,1.0,sub_240
sub_240_200710,sub_240_200710,1.000000e+10,1.0,sub_240
sub_240_200711,sub_240_200711,1.000000e+10,1.0,sub_240


In [33]:
# Change obd group name
for i in range(0, 3):
    obd.iloc[i, 3] = obd.iloc[i, 0][:7]
obd

,obsnme,obsval,weight,obgnme
bfr_066,bfr_066,1.000000e+10,1.0,bfr
bfr_068,bfr_068,1.000000e+10,1.0,bfr
bfr_147,bfr_147,1.000000e+10,1.0,bfr
sub_225_200301,sub_225_200301,1.000000e+10,1.0,sub_225
sub_225_200302,sub_225_200302,1.000000e+10,1.0,sub_225
...,...,...,...,...
sub_240_200708,sub_240_200708,1.000000e+10,1.0,sub_240
sub_240_200709,sub_240_200709,1.000000e+10,1.0,sub_240
sub_240_200710,sub_240_200710,1.000000e+10,1.0,sub_240
sub_240_200711,sub_240_200711,1.000000e+10,1.0,sub_240


#### 2.3. Import measured data

In [35]:
stf_obd = pd.read_csv('streamflow.obd',
                       sep='\t',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
stf_obd = stf_obd['1/1/2003': '12/31/2007']

bfr_obd = pd.read_csv('baseflow_ratio.obd',
                       sep='\s+',
                       index_col = 0,
                       header=None,
                     )
bfr_obd


,1
0,
bfr_068,0.94
bfr_066,0.92
bfr_147,0.60


In [36]:
bfr_obd.iloc[:, 0].tolist()

[0.94, 0.92, 0.6]

In [37]:
# Get sub list based on obd order
sub_order = []
for i in obd.obgnme.tolist():
    if i not in sub_order:
        sub_order.append(i)
sub_order

['    bfr', 'sub_225', 'sub_240']

In [38]:
# get total list from each sub obd, delete na vals
tot_obd = []
for i in sub_order[1:]:
    tot_obd += stf_obd[i].dropna().tolist()
len(tot_obd)
tot_obd = bfr_obd.iloc[:, 0].tolist() + tot_obd
tot_obd

[0.94,
 0.92,
 0.6,
 74.42,
 127.73,
 321.48,
 133.87,
 72.26,
 30.13,
 33.61,
 14.86,
 38.82,
 28.88,
 101.25,
 95.48,
 376.81,
 764.41,
 562.9,
 538.83,
 394.84,
 176.93,
 119.35,
 90.74,
 59.73,
 85.42,
 97.93,
 119.77,
 106.71,
 175.46,
 266.26,
 680.6,
 423.1,
 159.2,
 114.67,
 64.61,
 43.87,
 26.0,
 89.06,
 80.96,
 113.0,
 116.57,
 141.93,
 374.4,
 381.06,
 137.93,
 66.29,
 56.48,
 30.68,
 59.33,
 55.64,
 119.79,
 312.62,
 247.93,
 563.81,
 828.0,
 364.03,
 150.53,
 66.55,
 51.57,
 17.0,
 68.41,
 87.97,
 127.87,
 191.15,
 316.17,
 449.39,
 379.94,
 348.33,
 260.6,
 189.7,
 156.52,
 127.89,
 98.61,
 110.5,
 163.37,
 413.53,
 676.98,
 686.13,
 570.57,
 480.18,
 317.79,
 250.6,
 193.11,
 155.77,
 127.42,
 114.55,
 157.25,
 190.17,
 339.61,
 439.54,
 550.22,
 501.62,
 300.19,
 212.55,
 165.45,
 136.59,
 104.52,
 108.22,
 144.1,
 213.02,
 275.69,
 321.97,
 411.81,
 456.61,
 347.92,
 236.67,
 180.89,
 148.43,
 126.43,
 148.07,
 252.6,
 432.09,
 445.61,
 524.87,
 694.1,
 445.97,
 300.24

In [39]:
obd.loc[:, 'obsval'] = tot_obd
obd

,obsnme,obsval,weight,obgnme
bfr_066,bfr_066,0.94,1.0,bfr
bfr_068,bfr_068,0.92,1.0,bfr
bfr_147,bfr_147,0.60,1.0,bfr
sub_225_200301,sub_225_200301,74.42,1.0,sub_225
sub_225_200302,sub_225_200302,127.73,1.0,sub_225
...,...,...,...,...
sub_240_200708,sub_240_200708,180.87,1.0,sub_240
sub_240_200709,sub_240_200709,152.49,1.0,sub_240
sub_240_200710,sub_240_200710,118.21,1.0,sub_240
sub_240_200711,sub_240_200711,105.10,1.0,sub_240


### 4. Export control file

In [40]:
pst.control_data.noptmax=0

In [41]:
pst.model_command = 'python forward.py'

In [42]:
pst.write('okvg_pest2.pst')

noptmax:0, npar_adj:84, nnz_obs:123


In [46]:
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
awc_147,awc_147,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
awc_227,awc_227,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
awc_240,awc_240,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
awc_243,awc_243,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
chd_147,chd_147,log,factor,1.001000,0.001,10.0,str,1.0,-1.0,1
...,...,...,...,...,...,...,...,...,...,...
sy04,sy04,log,factor,173.522220,5.000,500.0,sy,1.0,0.0,1
sy05,sy05,log,factor,500.000000,10.000,1000.0,sy,1.0,0.0,1
sy06,sy06,log,factor,800.000000,15.000,1500.0,sy,1.0,0.0,1
sy07,sy07,log,factor,1000.000000,50.000,5000.0,sy,1.0,0.0,1
